# Цель исследования

Определить уровень потребительской лояльности клиентов в России и рассчитать показатель NPS по результатам опроса

# План работы
	1.	Загрузить данные из SQLite
	2.	Проверить и подготовить данные
	3.	Разделить клиентов на группы (сторонники, нейтралы, критики)
	4.	Рассчитать NPS
	5.	Построить дашборд с результатами
    
# Описание данных
	•	user — данные об ответах пользователей (основная таблица)
	•	location — данные о регионах
	•	age_segment — сегменты по возрасту
	•	traffic_segment — сегменты по объёму потребления трафика
	•	lifetime_segment — сегменты по длительности пользования услугами

In [1]:
import os
import pandas as pd
import numpy as np
 
from sqlalchemy import create_engine

In [2]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [3]:
query = """
SELECT u.user_id,
       u.lt_day,
       CASE 
           WHEN u.lt_day <= 365 THEN 'new'
           ELSE 'old'
       END AS is_new,
       u.age,
       CASE 
           WHEN u.gender_segment = 1 THEN 'woman'
           WHEN u.gender_segment = 0 THEN 'man'
           ELSE 'unknown'
       END as gender_segment,
       u.os_name,
       u.cpe_type_name,
       l.country,
       l.city,
       a.title AS age_segment,
       t.title AS traffic_segment,
       lt.title AS lifetime_segment,
       CASE 
           WHEN u.nps_score BETWEEN 9 AND 10 THEN 'cторонники'
           WHEN u.nps_score BETWEEN 7 AND 8 THEN 'нейтралы'
           WHEN u.nps_score BETWEEN 0 AND 6 THEN 'критики'
       END AS nps_group
FROM user AS u
LEFT JOIN location AS l ON u.location_id = l.location_id
LEFT JOIN age_segment AS a ON u.age_gr_id = a.age_gr_id
LEFT JOIN traffic_segment AS t ON u.tr_gr_id = t.tr_gr_id
LEFT JOIN lifetime_segment AS lt ON u.lt_gr_id = lt.lt_gr_id
"""

In [4]:
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_group
0,A001A2,2320,old,45.0,woman,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,cторонники
1,A001WF,2344,old,53.0,man,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,cторонники
2,A003Q7,467,old,57.0,man,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,cторонники


In [5]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

Дашборд:
https://public.tableau.com/views/NPS_17569215709190/NPS-_1?:language=en-US&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link